<a href="https://colab.research.google.com/github/jorgemar723/3DObjectReconstruction/blob/main/notebooks/real_photos_reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Library imports and set up
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import trimesh

In [ ]:
#Convert to greyscale
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

sift = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

keypoints, descriptors = sift.detectAndCompute(gray, None)

img_keypoints = cv2.drawKeypoints(gray, keypoints, None,
                                   flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

In [ ]:
# Create brute force matcher
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

# Match descriptors between two images
matches = bf.knnMatch(descriptors1, descriptors2, k=2)

# Apply ratio test (Lowe's ratio test)
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

# Draw matches
img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2,
                              good_matches, None,
                              flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)


In [ ]:
# Extract matched point coordinates
points1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches])
points2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches])

# Camera intrinsics (adjust for image size)
h, w = gray1.shape
focal_length = w  # rough estimate
principal_point = (w/2, h/2)

K = np.array([[focal_length, 0, principal_point[0]],
              [0, focal_length, principal_point[1]],
              [0, 0, 1]], dtype=np.float32)

# Find essential matrix using RANSAC
E, mask = cv2.findEssentialMat(points1, points2, K, method=cv2.RANSAC, prob=0.999, threshold=1.0)

# Recover rotation and translation
_, R, t, mask_pose = cv2.recoverPose(E, points1, points2, K)


In [ ]:
# Create projection matrices
P1 = K @ np.hstack([np.eye(3), np.zeros((3, 1))])
P2 = K @ np.hstack([R, t])

# Triangulate points
inliers = mask.ravel().tolist()
points1_tri = points1[inliers].T
points2_tri = points2[inliers].T
points_4d = cv2.triangulatePoints(P1, P2, points1_tri, points2_tri)

# Convert to 3D
points_3d = points_4d[:3] / points_4d[3]
points_3d = points_3d.T


In [ ]:
# Create 3D plot
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(points_3d[:, 0], points_3d[:, 1], points_3d[:, 2],
           c='blue', marker='o', s=50, alpha=0.6)

# Plot camera positions
ax.scatter(0, 0, 0, c='red', marker='^', s=200, label='Camera 1')
ax.scatter(t[0], t[1], t[2], c='green', marker='^', s=200, label='Camera 2')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
plt.show()


In [ ]:
import trimesh

# Create point cloud
points_mesh = trimesh.points.PointCloud(vertices=points_3d)

# Export
points_mesh.export('reconstruction.ply')
